In [1]:
import pandas as pd

In [2]:
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
excel_path = os.path.join(parent_dir, "data", "Gen_AI_Dataset.xlsx")

In [3]:
train_df = pd.read_excel(excel_path, sheet_name="Train-Set")
print("Train Set Loaded!")
train_df.head()

Train Set Loaded!


,Query,Assessment_url
0,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
1,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
2,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
3,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
4,I am hiring for Java developers who can also c...,https://www.shl.com/products/product-catalog/v...


In [4]:
import os, sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [5]:
CHROMA_DB_DIR1 = os.path.join(project_root, 'app', 'chroma_shl_db_1')
CHROMA_DB_DIR2 = os.path.join(project_root, 'app', 'chroma_shl_db_2')

In [6]:
from app.utils.get_retriever import get_retriever
ground_truth = (
    train_df.groupby("Query")["Assessment_url"].apply(list).to_dict()
)
print(f"Total unique queries: {len(ground_truth)}")

retriever1 =get_retriever(CHROMA_DB_DIR1)
retriever2 =get_retriever(CHROMA_DB_DIR2)

c:\Users\Ankit\Desktop\SHL-GenAI-Assessment-Recommendation\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Total unique queries: 10


c:\Users\Ankit\Desktop\SHL-GenAI-Assessment-Recommendation\app\utils\get_retriever.py:8: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectordb = Chroma(


In [ ]:
recalls = []
"""
Note: The scraped catalog uses 'https://www.shl.com/products/', 
while the training data has 'https://www.shl.com/solutions/products/'. 
This caused mismatches, so I removed/replaced 'solutions' in URLs. 
Both forms are valid and refer to the same SHL product pages.
"""

for query, gt_urls in ground_truth.items():
    docs = retriever1.invoke(query)

    retrieved = []
    for doc in docs:
        for line in doc.page_content.splitlines():
            if "url:" in line.lower():
                url = line.split(":", 1)[1].strip().lower()
                url = url.replace("https://www.shl.com/solutions/products/", "https://www.shl.com/products/")
                if url.endswith("/"):
                    url = url[:-1]
                retrieved.append(url)

    matches = 0
    for gt in gt_urls:
        gt = gt.strip().lower()
        if gt.endswith("/"):
            gt = gt[:-1]
        gt = gt.replace("https://www.shl.com/solutions/products/", "https://www.shl.com/products/")

        for ru in retrieved:
            if gt in ru or ru in gt or gt.split("/")[-1].split("-")[0] in ru:
                matches += 1
                break

    recall = matches / len(gt_urls) if len(gt_urls) else 0
    recalls.append(recall)

mean_recall = sum(recalls) / len(recalls)
print(f"\n Mean Recall@10: {mean_recall:.3f}")



 Mean Recall@10: 0.293


In [ ]:
recalls = []
"""
Note: The scraped catalog uses 'https://www.shl.com/products/', 
while the training data has 'https://www.shl.com/solutions/products/'. 
This caused mismatches, so I removed/replaced 'solutions' in URLs. 
Both forms are valid and refer to the same SHL product pages.
"""

for query, gt_urls in ground_truth.items():
    docs = retriever2.invoke(query)

    retrieved = []
    for doc in docs:
        for line in doc.page_content.splitlines():
            if "url:" in line.lower():
                url = line.split(":", 1)[1].strip().lower()
                url = url.replace("https://www.shl.com/solutions/products/", "https://www.shl.com/products/")
                if url.endswith("/"):
                    url = url[:-1]
                retrieved.append(url)

    matches = 0
    for gt in gt_urls:
        gt = gt.strip().lower()
        if gt.endswith("/"):
            gt = gt[:-1]
        gt = gt.replace("https://www.shl.com/solutions/products/", "https://www.shl.com/products/")

        for ru in retrieved:
            if gt in ru or ru in gt or gt.split("/")[-1].split("-")[0] in ru:
                matches += 1
                break

    recall = matches / len(gt_urls) if len(gt_urls) else 0
    recalls.append(recall)

mean_recall = sum(recalls) / len(recalls)
print(f"\n Mean Recall@10: {mean_recall:.3f}")



 Mean Recall@10: 0.433
